In [1]:
import pandas as pd
import dbconfig as dbConf

In [31]:
def Patten_GetNumber(max_val, f1, f2, f3, f4):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = (f"""
             select sum(case when row_num = 1 then num end) as n1
                  , sum(case when row_num = 2 then num end) as n2
                  , sum(case when row_num = 3 then num end) as n3
                  , sum(case when row_num = 4 then num end) as n4
                  , sum(case when row_num = 5 then num end) as n5
                  , sum(case when row_num = 6 then num end) as n6
                  , '' as bin
                  , sum(case when row_num = 1 then seq end) as f1
                  , sum(case when row_num = 2 then seq end) as f2
                  , sum(case when row_num = 3 then seq end) as f3
                  , sum(case when row_num = 4 then seq end) as f4
                  , sum(case when row_num = 5 then seq end) as f5
                  , sum(case when row_num = 6 then seq end) as f6
               from (
             select num
                  , @rownum:=@rownum+1 as row_num
                  , seq
               from (
                    select seq, num   
                      from (select 3 as seq, num
                              from in_list 
                             where seq = {max_val} 
                               and val = 3 
                             order by rand() 
                             limit {f1}
                           ) as t3
                    union all
                    select seq, num   
                      from (select 5 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 5 
                             order by rand() 
                             limit {f2}
                           ) as t5 
                    union all
                    select seq, num   
                      from (select 10 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 10
                             order by rand() 
                             limit {f3}
                           ) as t10
                    union all
                    select seq, num   
                    from (select 30 as seq, num
                            from in_list 
                           where seq = {max_val}
                             and val = 30 
                           order by rand() 
                           limit {f4}
                         ) as t30
                     order by num 
                 ) as a
                 , (SELECT @rownum:=0) TMP
                 ) as b 
             ;
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [7]:
def GetPatten():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    #print("접속완료")

    sql = ("""
           select stan
                , 3cnt
                , 5cnt
                , 10cnt
                , 30cnt
             from in_list_2
            order by 1 desc
            limit 15
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## 패턴

In [311]:
df = GetPatten()

stan  = list(range(0,1))
cnt3  = list(range(0,1))
cnt5  = list(range(0,1))
cnt10 = list(range(0,1))
cnt30 = list(range(0,1))

for df_stan, df_cnt3, df_cnt5, df_cnt10, df_cnt30 in zip(df['stan'], df['3cnt'], df['5cnt'], df['10cnt'], df['30cnt']):
    stan.append(int(df_stan))
    cnt3.append(int(df_cnt3))
    cnt5.append(int(df_cnt5))
    cnt10.append(int(df_cnt10))
    cnt30.append(int(df_cnt30))

df

,stan,3cnt,5cnt,10cnt,30cnt
0,1013,3,0,1,2
1,1012,3,1,1,1
2,1011,1,1,2,2
3,1010,2,0,3,1
4,1009,4,0,1,1
5,1008,2,1,0,3
6,1007,1,2,1,2
7,1006,3,1,1,1
8,1005,3,1,2,0
9,1004,1,1,3,1


## 데이터 뽑기

In [305]:
# 최신 패턴으로 결과 뽑기
data = Patten_GetNumber(stan[1],0,0,0,0)

f1 = 0
f2 = 2
f3 = 3
f4 = 1

for i in range(0, 5):
    data = data.append(Patten_GetNumber(stan[1]+1, f1, f2, f3, f4), ignore_index = True)
    data['num_sum'] = data['n1'] + data['n2'] + data['n3'] + data['n4'] + data['n5'] + data['n6'] 
    
data

,n1,n2,n3,n4,n5,n6,bin,f1,f2,f3,f4,f5,f6,num_sum
0,None,None,None,None,None,None,,None,None,None,None,None,None,NaN
1,3,23,24,30,39,40,,30,5,10,10,10,5,159
2,7,17,23,25,27,43,,10,10,5,5,10,30,142
3,7,10,15,17,23,37,,10,30,5,10,5,10,109
4,6,7,15,19,27,44,,30,10,5,10,10,5,118
5,13,15,16,25,33,37,,10,5,10,5,30,10,139


In [268]:
data2 = Patten_GetNumber(stan[1],0,0,0,0)

data2 = data2.append(Patten_GetNumber(stan[1]+1, f1, f2, f3, f4), ignore_index = True)

data2['num_sum'] = data2['n1'] + data2['n2'] + data2['n3'] + data2['n4'] + data2['n5'] + data2['n6'] 


data2

,n1,n2,n3,n4,n5,n6,bin,f1,f2,f3,f4,f5,f6,num_sum
0,None,None,None,None,None,None,,None,None,None,None,None,None,NaN
1,13,19,23,30,33,40,,10,10,5,10,30,5,158


-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------

In [33]:
def sql_direct(sql_list):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = sql_list

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [310]:
sql = """
select a.*
     , ifnull(sum_num - (select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) from temp b where b.seq = (a.seq + 1)),0) as diff
  from (
        select a.seq
             , a.n1, a.n2, a.n3, a.n4, a.n5, a.n6
             , (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
          from temp a
       ) a
 where seq > 990
 order by 1 desc
       
"""

sql_data = sql_direct(sql)

sql_data

,seq,n1,n2,n3,n4,n5,n6,sum_num,diff
0,1014,3,11,14,18,26,27,99,0
1,1013,21,22,26,34,36,41,180,81
2,1012,5,11,18,20,35,45,134,-46
3,1011,1,9,12,26,35,38,121,-13
4,1010,9,12,15,25,34,36,131,10
5,1009,15,23,29,34,40,44,185,54
6,1008,9,11,30,31,41,44,166,-19
7,1007,8,11,16,19,21,25,100,-66
8,1006,8,11,15,16,17,37,104,4
9,1005,8,13,18,24,27,29,119,15
